In [5]:
!pip install -U langchain langchain-community langchain-core langchain-text-splitters pypdf chromadb gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Task 1  - loading the documents

In [6]:
from langchain.document_loaders import PyPDFLoader

In [7]:
pdf_path = "/content/A_Comprehensive_Review_of_Low_Rank_Adaptation_in_Large_Language_Models_for_Efficient_Parameter_Tuning-1.pdf"

#LOAD PDF WITH PyPDFloader

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print("First page content :\n")
print(documents[0].page_content[:500])
print("\n Total pages loaded:" , len(documents))

First page content :

A Comprehensive Review of Low-Rank
Adaptation in Large Language Models for
Efficient Parameter Tuning
September 10, 2024
Abstract
Natural Language Processing (NLP) often involves pre-training large
models on extensive datasets and then adapting them for specific tasks
through fine-tuning. However, as these models grow larger, like GPT-3
with 175 billion parameters, fully fine-tuning them becomes computa-
tionally expensive. We propose a novel method called LoRA (Low-Rank
Adaptation) that signifi

 Total pages loaded: 11


**Task 2 ( text splitter )**

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
#create text splitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)

#split into chunks
splitted_docs = text_splitter.split_documents(documents)

print(f" Total chunks created: {len(splitted_docs)}")
print("\n Sample chunk (first one ):\n ")
print(splitted_docs[0].page_content[:500])

 Total chunks created: 38

 Sample chunk (first one ):
 
A Comprehensive Review of Low-Rank
Adaptation in Large Language Models for
Efficient Parameter Tuning
September 10, 2024
Abstract
Natural Language Processing (NLP) often involves pre-training large
models on extensive datasets and then adapting them for specific tasks
through fine-tuning. However, as these models grow larger, like GPT-3
with 175 billion parameters, fully fine-tuning them becomes computa-
tionally expensive. We propose a novel method called LoRA (Low-Rank
Adaptation) that signifi


**tasks 3 ( generate embeddings )**

In [11]:
!pip install -q sentence-transformers

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings

#initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#generate embedding for our chunks
docs_embeddings = embedding_model.embed_documents([doc.page_content for doc in splitted_docs])

print("Number of embeddings:", len(docs_embeddings))
print("size of each embedding vector:" , len(docs_embeddings[0]))
print("\n Sample embedding (frist 10 numbers):\n",  docs_embeddings[0][:10])

/tmp/ipython-input-764819094.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of embeddings: 38
size of each embedding vector: 384

 Sample embedding (frist 10 numbers):
 [-0.046186283230781555, -0.09967904537916183, 0.015307209454476833, 0.048012156039476395, 0.06438607722520828, 0.039436426013708115, -0.0393260158598423, 0.018974635750055313, 0.0039568329229950905, -0.06212152540683746]


**Tasks 4 ( Chroma Vector DB)**

In [13]:
!pip install -q chromadb

In [14]:
from langchain.vectorstores import Chroma

In [15]:
#create Chroma vector DB
vectordb = Chroma.from_documents(
    documents = splitted_docs,
    embedding = embedding_model,
    collection_name = "research_paper"
)

print("Number of documents in vector DB : ", vectordb._collection.count())

Number of documents in vector DB :  38


**Task 5 (Retriever)**

In [17]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [18]:
# create retriver from vector DB
retriever = vectordb.as_retriever(search_type= "similarity", search_kwargs={"k":3})

#Test : fetch top 3 chunks for a query
query = "what this paper is talking about ?"
retrieved_docs = retriever.get_relevant_documents(query)

for i , doc in enumerate(retrieved_docs):
  print(f"--- chunk {i+1} ---")
  print(doc.page_content[:500])
  print("\n")

--- chunk 1 ---
ing it as a low-rank decomposition, W0 + ∆W = W0 + BA, where B ∈ Rd×r,
A ∈ Rr×k, and the rank r ≪ min(d, k). During training, W0 is fixed, and A
and B are the trainable parameters. Both W0 and ∆W = BA are multiplied
with the input, and their respective outputs are summed element-wise. Thus,
for h = W0x, our updated forward pass becomes:
h = W0x + ∆W x= W0x + BAx
We illustrate this reparametrization in Figure 1. We initializeA with random
Gaussian values and set B to zero, meaning ∆ W = BA is zer


--- chunk 2 ---
The principles outlined here apply generally to dense layers in neural networks,
although we focus on specific weights in Transformer language models, as these
models serve as the central example in our experiments.
4.1 Low-Rank Parameterized Update Matrices
Neural networks contain numerous dense layers that perform matrix multipli-
cation, and the weight matrices in these layers typically have a full rank. When
adapting to a particular task, it shows that pre-

/tmp/ipython-input-1924485184.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


**`Task 6 ( Gardio QA Bot ) `**



In [23]:
!pip install -q transformers sentence-transformers gradio langchain


In [24]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Load model and tokenizer
model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Create Hugging Face pipeline
hf_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)

# Wrap in LangChain LLM
llm = HuggingFacePipeline(pipeline=hf_pipeline)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
/tmp/ipython-input-469883060.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [25]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True  # optional: show which chunks were used
)


In [27]:
def answer_question_safe(question):
    docs = retriever.get_relevant_documents(question)
    if len(docs) == 0:
        return "No relevant information found in the PDF."

    # Combine chunks but limit total length
    context = " ".join([doc.page_content for doc in docs])
    context = context[:3000]  # truncate to 3000 chars to avoid model overflow

    # Construct prompt for Hugging Face model
    prompt = f"Answer the question based on the context below.\n\nContext: {context}\n\nQuestion: {question}\nAnswer:"

    return llm(prompt)


In [28]:
iface = gr.Interface(
    fn=answer_question_safe,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question..."),
    outputs="text",
    title="Research Paper QA Bot",
    description="Ask questions from your uploaded PDF"
)

iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://474163e0b8af547c07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
